# 神经网络基础
## 层和块
如何通过继承nn.Module类来完成比较灵活的构造

In [3]:
# 之前多层感知机的实现
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

OSError: [WinError 127] 找不到指定的程序。 Error loading "D:\conda\envs\A1\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

In [5]:
# 自定义块
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数。
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)  # 实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

NameError: name 'nn' is not defined

In [8]:
# 顺序块，实现两个关键函数：1、一种将块逐个追加到列表中的函数；2、一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

NameError: name 'nn' is not defined

In [10]:
# 在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)


NameError: name 'nn' is not defined

In [12]:
# 随意混搭
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

NameError: name 'nn' is not defined

## 参数管理
访问参数、参数初始化、共享两个层的参数

In [13]:
# 单隐藏层的感知机
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

print(net[2].state_dict()) # 参数访问

OSError: [WinError 127] 找不到指定的程序。 Error loading "D:\conda\envs\A1\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

In [14]:
print(type(net[2].bias)) # 第三个神经网络层提取偏置
print(net[2].bias)
print(net[2].bias.data)

NameError: name 'net' is not defined

In [15]:
net[2].weight.grad == None

NameError: name 'net' is not defined

In [17]:
# 访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

NameError: name 'net' is not defined

In [18]:
net.state_dict()['2.bias'].data

NameError: name 'net' is not defined

In [19]:
# 生成块
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

NameError: name 'nn' is not defined

In [20]:
print(rgnet) # 查看一下结构

NameError: name 'rgnet' is not defined

In [21]:
rgnet[0][1][0].bias.data # 访问参数

NameError: name 'rgnet' is not defined

In [22]:
def init_normal(m):  # 参数初始化
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

NameError: name 'net' is not defined

In [23]:
def init_constant(m): # 另外一种参数初始化
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

NameError: name 'net' is not defined

In [24]:
# 某些块应用不同的初始化方法
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

NameError: name 'net' is not defined

In [25]:
# 自定义参数初始化方法
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

NameError: name 'net' is not defined

In [26]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]  # 直接设置参数的值

NameError: name 'net' is not defined

In [27]:
# 层与层之间参数共享，首先需要定义一个共享层
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

NameError: name 'nn' is not defined

## 自定义层

In [29]:
# 不带参数的层
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

OSError: [WinError 127] 找不到指定的程序。 Error loading "D:\conda\envs\A1\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

In [30]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))   

NameError: name 'CenteredLayer' is not defined

In [31]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer()) # 将层作为组件合并到更复杂的模型中
Y = net(torch.rand(4, 8))
Y.mean()

NameError: name 'nn' is not defined

In [32]:
# 带参数的层，该层需要输入参数：in_units和units，分别表示输入数和输出数
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

linear = MyLinear(5, 3)
linear.weight

NameError: name 'nn' is not defined

In [33]:
linear(torch.rand(2, 5))

NameError: name 'linear' is not defined

In [34]:
# 使用自定义层构建模型
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

NameError: name 'nn' is not defined

In [35]:
## 读写文件，直接调用load和save函数分别读写它们
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')
x2 = torch.load('x-file')
x2

OSError: [WinError 127] 找不到指定的程序。 Error loading "D:\conda\envs\A1\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

In [36]:
# 列表存储
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

NameError: name 'torch' is not defined

In [37]:
# 字典存储
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

NameError: name 'x' is not defined

In [38]:
# 保存模型参数
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)
torch.save(net.state_dict(), 'mlp.params')

NameError: name 'nn' is not defined

In [ ]:
# 恢复模型，实例化了原始多层感知机模型的一个备份
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()
Y_clone = clone(X)
Y_clone == Y